In [1]:
import time

from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
import polars as pl

from plotly.subplots import make_subplots
from result import Result

from raft.io import read_file

from raft.utils.log import logger

from raft.tasks.message_parsing.data_types import MessageParseData

In [2]:
from raft.services.sirius_messages import (
    ImageCaptureMessage,
    SeabirdCTDMessage,
    AanderaaCTDMessage,
    EcopuckMessage,
    ParosciPressureMessage,
    TeledyneDVLMessage,
    LQModemMessage,
    EvologicsModemMessage,
    BatteryMessage,
    ThrusterMessage,
)

from raft.services.sirius_messages import (
    parse_message_header,
    parse_image_message,
    parse_seabird_ctd_message,
    parse_aanderaa_ctd_message,
    parse_ecopuck_message,
    parse_parosci_pressure_message,
    parse_teledyne_dvl_message,
    parse_lq_modem_message,
    parse_evologics_modem_message,
    parse_battery_message,
    parse_thruster_message,
)

### Plot LQ Modem 

In [3]:
def plot_lqmodem_scatter(data: pl.DataFrame) -> None:
    """TODO"""
    fig = px.scatter_mapbox(
        data,
        lat="latitude",
        lon="longitude",
        color="timestamp",
        size_max=15,
        zoom=16,
        height=600,
        hover_data={"time"},
    )
    fig.update_layout(height=800, width=1000, mapbox_style="open-street-map")
    return fig


def plot_evologics_scatter(data: pl.DataFrame) -> None:
    """TODO"""
    fig = px.scatter_mapbox(
        data,
        lat="target_latitude",
        lon="target_longitude",
        color="timestamp",
        size_max=15,
        zoom=16,
        height=600,
        hover_data={"target_depth"},
    )
    fig.update_layout(height=800, width=1000, mapbox_style="open-street-map")
    return fig


logger.info(data["lqmodem"].head())

plot_usbl = True
if plot_usbl:
    figs["lqmodem"] = plot_lqmodem_scatter(data["lqmodem"])
    figs["lqmodem"].show()

NameError: name 'data' is not defined

### Plot DVL data

In [ ]:
def plot_dvl_timeseries(data: pl.DataFrame) -> None:
    """TODO"""
    fig = make_subplots(
        rows=10,
        cols=1,
        subplot_titles=(
            "DMG-X",
            "DMG-Y",
            "DMG-Z",
            "Velocity X",
            "Velocity Y",
            "Velocity Z",
            "Altitude",
            "Heading",
            "Pitch",
            "Roll",
        ),
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["dmg_x"],
        ),
        row=1,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["dmg_y"],
        ),
        row=2,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["dmg_z"],
        ),
        row=3,
        col=1,
    )

    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["velocity_x"],
        ),
        row=4,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["velocity_y"],
        ),
        row=5,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["velocity_z"],
        ),
        row=6,
        col=1,
    )

    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["altitude"],
        ),
        row=7,
        col=1,
    )

    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["heading"],
        ),
        row=8,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["pitch"],
        ),
        row=9,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["roll"],
        ),
        row=10,
        col=1,
    )

    fig.update_layout(height=2000, width=1000, title_text="DVL Timeseries")

    return fig


if plot_dvl:
    figs["dvl_timeseries"] = plot_dvl_timeseries(data["dvl"])
    figs["dvl_timeseries"].show()

### Plot thruster data

In [ ]:
def plot_thruster_timeseries(data: pl.DataFrame, title: str) -> None:
    """TODO"""
    fig = make_subplots(
        rows=4, cols=1, subplot_titles=("RPM", "Current", "Voltage", "Temperature")
    )

    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["rpm"],
        ),
        row=1,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["current"],
        ),
        row=2,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["voltage"],
        ),
        row=3,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["temperature"],
        ),
        row=4,
        col=1,
    )

    fig.update_layout(height=1000, width=1000, title_text=title)

    return fig


if plot_thrusters:
    pass

### Plot pressure data

In [ ]:
def plot_pressure_timeseries(data: pl.DataFrame) -> None:
    """TODO"""
    fig = make_subplots(rows=1, cols=1, subplot_titles=("Depth",))

    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["depth"],
        ),
        row=1,
        col=1,
    )
    fig.update_layout(height=400, width=1000, title_text="Pressure Sensor")

    return fig


if plot_pressure:
    figs["pressure"] = plot_pressure_timeseries(data["pressure"])
    figs["pressure"].show()

### Plot image capture data

In [ ]:
def plot_image_timeseries(data: pl.DataFrame) -> None:
    """TODO"""
    fig = make_subplots(rows=2, cols=1, subplot_titles=("Left camera", "Right camera"))

    left: pl.DataFrame = data.filter(pl.col("label").str.ends_with("LC16"))
    right: pl.DataFrame = data.filter(
        pl.col("label").str.ends_with("RM16") | pl.col("label").str.ends_with("RC16")
    )

    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=left["exposure"],
        ),
        row=1,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=right["exposure"],
        ),
        row=2,
        col=1,
    )
    fig.update_layout(height=600, width=1000, title_text="Image Exposure")

    return fig


if plot_image_data:
    figs["image"] = plot_image_timeseries(data["image"])
    figs["image"].show()

### Plot CTD data

In [ ]:
def plot_ctd_timeseries(data: pl.DataFrame) -> None:
    """TODO"""
    fig = make_subplots(
        rows=5,
        cols=1,
        subplot_titles=(
            "Conductivity",
            "Temperature",
            "Salinity",
            "Pressure",
            "Sound Velocity",
        ),
    )

    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["conductivity"],
        ),
        row=1,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["temperature"],
        ),
        row=2,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["salinity"],
        ),
        row=3,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["pressure"],
        ),
        row=4,
        col=1,
    )
    fig.append_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["sound_velocity"],
        ),
        row=5,
        col=1,
    )

    fig.update_layout(height=1000, width=1000, title_text="CTD Data")

    return fig


plot_ctd: bool = True
if plot_ctd:
    figs["ctd"] = plot_ctd_timeseries(data["ctd"])
    figs["ctd"].show()